In [15]:
import os
import pandas as pd
from blast_utils import make_diamond_db, diamond_alignment

In [35]:
os.makedirs('ref_databases', exist_ok=True)
#make diamond reference database for BLAST
make_diamond_db('../../splits/task1/fastas/train_swissprot.fasta', 'ref_databases/train_swissprot') 

diamond v2.1.9.163 (C) Max Planck Society for the Advancement of Science, Benjamin Buchfink, University of Tuebingen
Documentation, support and updates available at http://www.diamondsearch.org
Please cite: http://dx.doi.org/10.1038/s41592-021-01101-x Nature Methods (2021)

#CPU threads: 384
Scoring parameters: (Matrix=BLOSUM62 Lambda=0.267 K=0.041 Penalties=11/1)
Database input file: ../../splits/task1/fastas/train_swissprot.fasta
Opening the database file...  [0.014s]
Loading sequences...  [0.203s]
Masking sequences...  [0.074s]
Writing sequences...  [0.065s]
Hashing sequences...  [0.018s]
Loading sequences...  [0s]
Writing trailer...  [0.003s]
Closing the input file...  [0s]
Closing the database file...  [0.021s]

Database sequences  184529
  Database letters  72738365
     Database hash  e1cfdedf246106b48c94446badba64a5
        Total time  0.402000s


In [36]:
ref_withEC = pd.read_csv('../../processed_data/protein2EC.csv')

,Entry,Entry Name,Sequence,EC number,Length,EC All,clusterRes30,clusterRes50,clusterRes70,clusterRes90,EC3,EC2,EC1
0,A0A009IHW8,ABTIR_ACIB9,MSLEQKKGADIISKILQIQNSIGKTTSPSTLKTKLSEISRKEQENA...,3.2.2.6,269,3.2.2.-; 3.2.2.6,A1AY86,A0A009IHW8,A0A009IHW8,A0A009IHW8,3.2.2,3.2,3
6,A0A061AE05,PAPSH_CAEEL,MLTPRDENNEGDAMPMLKKPRYSSLSGQSTNITYQEHTISREERAA...,2.7.1.25,654,2.7.1.25; 2.7.7.4,A9G7W0,O43252,A0A061AE05,A0A061AE05,2.7.1,2.7,2
7,A0A061AE05,PAPSH_CAEEL,MLTPRDENNEGDAMPMLKKPRYSSLSGQSTNITYQEHTISREERAA...,2.7.7.4,654,2.7.1.25; 2.7.7.4,A9G7W0,O43252,A0A061AE05,A0A061AE05,2.7.7,2.7,2
12,A0A072VDF2,CCR1_MEDTR,MPAATAAAAAESSSVSGETICVTGAGGFIASWMVKLLLEKGYTVRG...,1.2.1.44,342,1.2.1.-; 1.2.1.44,Q5FB34,Q6K9A2,Q9S9N9,A0A072VDF2,1.2.1,1.2,1
15,A0A075FBG7,ELS_MARVU,MSITFNLKIAPFSGPGIQRSKETFPATEIQITASTKSTMTTKCSFN...,4.2.3.131,580,4.2.3.131; 4.2.3.189; 4.2.3.190,A0A075FBG7,A0A075FBG7,A0A075FBG7,A0A075FBG7,4.2.3,4.2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
185498,Q9UYY9,PORC_PYRAB,MIEIRFHGRGGQGAVTAANILAEAAFIEGKYVQAFPFFGVERRGAP...,1.2.7.7,185,1.2.7.1; 1.2.7.7,O27772,O27772,O58411,O58411,1.2.7,1.2,1
185525,Q9V0Y5,MTAD_PYRAB,MVSLSVLLRGGLVLYGRDYKLTRADVLIEGDKIVEVKRNINKPADE...,3.5.4.28,425,3.5.4.28; 3.5.4.31,A4J675,B6YUF8,Q9V0Y5,Q9V0Y5,3.5.4,3.5,3
185526,Q9V0Y5,MTAD_PYRAB,MVSLSVLLRGGLVLYGRDYKLTRADVLIEGDKIVEVKRNINKPADE...,3.5.4.31,425,3.5.4.28; 3.5.4.31,A4J675,B6YUF8,Q9V0Y5,Q9V0Y5,3.5.4,3.5,3
185853,Q9ZEE1,PDRP_RICPR,MTKLIIHLVSDSSVQTAKHAANSALAQFTSIKQKLYHWPMIRNCEL...,2.7.11.32,273,2.7.11.32; 2.7.4.27,B3E5Z6,C4K0B5,Q4UNK5,A8EX71,2.7.11,2.7,2


In [37]:
entry2EC = dict(zip(ref_withEC['Entry'], ref_withEC['EC All']))

In [38]:
names= ['price_protein_test', 'promiscuous_protein_test', '30_protein_test', '30-50_protein_test']

os.makedirs('../results_summary/BLAST', exist_ok=True)

for name in names:
    #plt.subplot(2, 3, i+1)
    df = pd.read_csv(f'../../splits/task1/{name}.csv')
    results = diamond_alignment(f'../../splits/task1/fastas/{name}.fasta', 'ref_databases/train_swissprot')
    #turn indices into a column
    results['Entry'] = results.index
    #merge the two dataframes
    merged = pd.merge(df, results, on='Entry')
    #add the EC numbers
    merged['EC Predicted'] = merged['ref_entry_id'].map(entry2EC)
    #rename EC predicted to 0
    # if "EC All" not in merged.columns:
    #     merged['EC All'] = merged['EC number']

    # merged.drop(columns=['EC number'], inplace=True)

    merged = merged.rename(columns={'EC Predicted': 0})
    merged[['Entry', 'EC number', 0]].to_csv(f'../results_summary/BLAST/{name}_results_df.csv', index=False)

In [34]:
merged

,Entry,Entry Name,Sequence,EC number,Length,EC All,clusterRes30,clusterRes50,clusterRes70,clusterRes90,EC3,EC2,EC1,aln_coverage,max_id,ref_entry_id,0
0,Q0KBD2,DEND_CUPNH,MNVLITGGAGFLGLQLARLLLQRGTLNLDGQPVAIKRLTLLDVVAP...,1.1.1.410,324,1.1.1.410,P44094,Q0KBD2,Q0KBD2,Q0KBD2,1.1.1,1.1,1,99.0,46.5300,P44094,1.1.1.410
1,D4GYH5,AGLM_HALVD,MELSIIGSGYVGTTIAACFAELGHDVVNVDIDEDIVASLNDGQAPI...,1.1.1.22,430,1.1.1.22,O34862,D4GYH5,D4GYH5,D4GYH5,1.1.1,1.1,1,99.1,42.4148,O54068,1.1.1.22
2,P9WQC7,ADHB_MYCTU,MKTKGALIWEFNQPWSVEEIEIGDPRKDEVKIQMEAAGMCRSDHHL...,1.1.1.1,375,1.1.1.1,P80094,P9WQC7,P9WQC7,P9WQC7,1.1.1,1.1,1,100.0,46.9000,P80175,1.1.99.36
3,Q94AX4,DLD_ARATH,MAFASKFARSKTILSFLRPCRQLHSTPKSTGDVTVLSPVKGRRRLP...,1.1.2.4,567,1.1.2.4,Q12627,Q94AX4,Q94AX4,Q94AX4,1.1.2,1.1,1,86.7,40.0554,F1QXM5,1.1.2.4
4,B8MKR3,CYB2_TALSN,MARVLDAAEVAKHNTPESCWVILYGKVYDVTEFISSHPGGVKVILR...,1.1.2.3,496,1.1.2.3,B8MKR3,B8MKR3,B8MKR3,B8MKR3,1.1.2,1.1,1,84.1,31.8739,P00175,1.1.2.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
521,Q9WXX0,RBSA1_THEMA,MFPLLAFRGDRMEILKAKGIVKRFPGVVAVDNVDFEVYENEIVSLI...,7.5.2.7,520,7.5.2.7,A4WER4,Q9WXX0,Q9WXX0,Q9WXX0,7.5.2,7.5,7,97.8,48.0198,Q891M1,7.5.2.7
522,Q831L8,TAGH_ENTFA,MEKELKVRTKLLTKEYSLAQTRIDKLKTLFSVFQNKVPTFWALKGV...,7.5.2.4,447,7.5.2.4,Q03SI5,Q831L8,Q831L8,Q831L8,7.5.2,7.5,7,95.7,39.8112,Q9CH26,7.5.2.4
523,Q9C8T1,AB1I_ARATH,MSIRRPQIPRLLLQNVSCMRNAQQILRHVNVSLHDGGALVLTGTNG...,7.6.2.5,229,7.6.2.5,Q2G9A9,Q9C8T1,Q9C8T1,Q9C8T1,7.6.2,7.6,7,91.7,36.4966,Q2VZJ1,7.6.2.5
524,Q0A808,CCMA_ALKEH,MGDTALHLVAAPALEATGLQVARGGRPLFRGLGFRLARGGLLCVRG...,7.6.2.5,236,7.6.2.5,Q2G9A9,Q0A808,Q0A808,Q0A808,7.6.2,7.6,7,86.7,39.6219,Q46RX0,7.6.2.5
